In [12]:
#libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
import joypy
import requests
import json

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.experimental import enable_halving_search_cv  
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor


import tracemalloc
import time
import yellowbrick
from yellowbrick.regressor import ResidualsPlot, CooksDistance
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.api import qqplot

import lime 
from lime import lime_tabular
import shap
shap.initjs()
import re

import warnings
warnings.filterwarnings('ignore')

In [13]:
df = pd.read_csv('_HDB.csv')
y = df[['resale_price']]
X = df.drop(['resale_price','town', 'year'], axis=1)

town  flat_type  storey_range  area_sqm  lease_start  year  \
0             Yishun          1             0      68.0         1978  2020   
1           Clementi          1             0      67.0         1978  2019   
2            Hougang          0             0      47.0         2012  2017   
3         Queenstown          1             0      72.0         1970  2016   
4         Queenstown          1             0      70.0         1970  2016   
...              ...        ...           ...       ...          ...   ...   
124648  Central Area          2            16      94.0         2011  2015   
124649  Central Area          2            16      94.0         2011  2018   
124650  Central Area          2            16      94.0         2011  2020   
124651  Central Area          3            16     105.0         2011  2017   
124652  Central Area          2            16      94.0         2011  2016   

        school_dist  num_school_2km  hawker_dist  num_hawker_2km  ...  \
0          0.325531              20     0.174269               1  ...   
1          0.442763              15     0.312990               5  ...   
2          0.450176              23     0.793415               3  ...   
3          0.590311               9     0.202540              10  ...   
4          0.881604               9     0.294493              11  ...   
...             ...             ...          ...             ...  ...   
124648     0.154258               7     0.237236              11  ...   
124649     0.281474               9     0.280703              13  ...   
124650     0.234687               7     0.188984              11  ...   
124651     0.301776               9     0.235487              13  ...   
124652     0.301776               9     0.235487              13  ...   

        resale_price  region_East  region_North  region_North East  \
0           240000.0            0             1                  0   
1           315000.0            0             0                  0   
2           233000.0            0             0                  1   
3           357000.0            0             0                  0   
4           330000.0            0             0                  0   
...              ...          ...           ...                ...   
124648      910000.0            0             0                  0   
124649      999000.0            0             0                  0   
124650     1085000.0            0             0                  0   
124651     1108000.0            0             0                  0   
124652      938880.0            0             0                  0   

        region_West  model_Apartment  model_Maisonette  model_Model A  \
0                 0                0                 0              0   
1                 1                0                 0              0   
2                 0                0                 0              1   
3                 0                0                 0              0   
4                 0                0                 0              0   
...             ...              ...               ...            ...   
124648            0                0                 0              0   
124649            0                0                 0              0   
124650            0                0                 0              0   
124651            0                0                 0              0   
124652            0                0                 0              0   

        model_New Generation  model_Special  
0                          1              0  
1                          1              0  
2                          0              0  
3                          0              0  
4                          0              0  
...                      ...            ...  
124648                     0              1  
124649                     0              1  
124650                     0              1  
124651                  

In [15]:
def tracing_start():
    tracemalloc.stop()
    print("nTracing Status : ", tracemalloc.is_tracing())
    tracemalloc.start()
    print("Tracing Status : ", tracemalloc.is_tracing())
def tracing_mem():
    first_size, first_peak = tracemalloc.get_traced_memory()
    peak = first_peak/(1024*1024)
    print("Peak Size in MB - ", peak)

In [16]:
# Train Test Split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

tracing_start()
start = time.time()

# Initialising and fitting random forest regressor including tuned hyperparameters as given by Halving Gridsearch
rfr_grid = RandomForestRegressor(n_estimators = 500, n_jobs=-1, random_state=0, max_depth=20, 
                                 max_features='auto', min_samples_leaf=2, min_samples_split=15)

#Predict training data
rfr_grid.fit(X_train, y_train)
rfr_grid_score = rfr_grid.score(X_train, y_train)   
print("Training data score: ", rfr_grid_score)
tracing_mem()

tracing_start()
start = time.time()
# Getting ypred (or prediction) values on X_test set
rfr_grid_predicted_test = rfr_grid.predict(X_test)

# Getting score on test set and calculate mean absolute error, mean squared error and root mean-squared error
rfr_grid_test_score = r2_score(y_test, rfr_grid_predicted_test)
rfr_grid_mae = mean_absolute_error(y_test, rfr_grid_predicted_test)
rfr_grid_mse = mean_squared_error(y_test, rfr_grid_predicted_test)
rfr_grid_rmse = np.sqrt(mean_squared_error(y_test, rfr_grid_predicted_test))
print("Test data R\u00b2 score: %f" % (rfr_grid_test_score))
print("Test data Mean Squared Error: %f" % (rfr_grid_mse))
print("Test data Mean Absolute Error: %f" % (rfr_grid_mae))
print("Test data Root Mean Squared Absolute Error: %f" % (rfr_grid_rmse))
end = time.time()
print("time elapsed {} milli seconds".format((end-start)*1000))
tracing_mem()

nTracing Status :  False
Tracing Status :  True
Training data score:  0.9773352146352863
Peak Size in MB -  33.95822334289551
nTracing Status :  False
Tracing Status :  True
Test data R² score: 0.966593
Test data Mean Squared Error: 746858799.174613
Test data Mean Absolute Error: 20464.193058
Test data Root Mean Squared Absolute Error: 27328.717481
time elapsed 332.75771141052246 milli seconds
Peak Size in MB -  3.8535919189453125


In [18]:
# Produce a matrix for client data
client_data = [[1, 3, 68, 1986, 0.594128259344289, 15 ,0.22516102677612, 8 ,0.376763931855833,28, 1.5803380574408, 1, 1.1060523511997, 2, 0.274218362591398, 9, 9.26348665949998, 0,0,1,0,0,0,1,0,0], # Client 1
               [2, 2, 93, 2010, 0.173394270589029, 20 ,0.495748156781795, 1 ,0.801641910325716,3, 1.2895891636303, 1, 0.354436984819771, 23, 0.0441655994955788, 17, 13.6128567065317, 0,0,1,0,1,0,0,0,0], # Client 2
               [2, 0, 112, 1995, 0.18934288586454, 17 ,2.03316923897357, 0 ,0.466650001317567,11, 0.398785707885446, 6, 0.97903269235057, 6, 0.413983861488505, 16, 13.782717157755, 1,0,0,0,0,0,1,0,0]]  # Client 3
print('Resale price: $255,000.00, $440,000.00, $385,000.00')
print('Adj. inflation: $268,429.53, $463,172.52, $403,720.52')
# Show predictions
for i, price in enumerate(rfr_grid.predict(client_data)):
    print("Predicted selling price for Client {}'s home: ${:,.2f}".format(i+1, price)) 

Resale price: $255,000.00, $440,000.00, $385,000.00
Adj. inflation: $268,429.53, $463,172.52, $403,720.52
Predicted selling price for Client 1's home: $291,947.95
Predicted selling price for Client 2's home: $470,587.85
Predicted selling price for Client 3's home: $399,731.49
